In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [37]:
class Inception_aux(nn.Module):
    def __init__(self,in_channel,num_output):
        super(Inception_aux,self).__init__()
        
        self.aux = nn.Sequential(
            nn.AvgPool2d(kernel_size=5,stride=3),
            Conv(in_channel,128,kernel_size=1),
            nn.ReLU(),
            nn.Linear(4*4*128,1024),
            nn.Dropout(p=0.7),
            nn.Linear(1024,num_output)
        )
        
    def forward(self,x):
        x = self.aux(x)
        return x

In [38]:
class Conv(nn.Module):
    def __init__(self,in_channel, out_channel,**kwargs):
        super(Conv,self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(in_channel,out_channel,**kwargs),
            nn.BatchNorm2d(out_channel),
            nn.ReLU()
        )

In [39]:
class InceptionV1(nn.Module):
    
    def __init__(self,in_channel,out_1,in_3,out_3,in_5,out_5,out_pooling):
        super(InceptionV1,self).__init__()
        
        self.branch_1 = Conv(in_channel,out_1,kernel_size=1)
        self.branch_2 = nn.Sequential(
            Conv(in_channel,in_3,kernel_size=1),
            Conv(in_3,out_3,kernel_size=3)
        )
        self.branch_3 = nn.Sequential(
            Conv(in_channel,in_5,kernel_size=1),
            Conv(in_5,out_5,kenel_size=5)
        )
        self.branch_4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3,stride=1),
            Conv(in_channel,out_pooling,kernel_size=1)
        )
        
    def forward(self,x):
        x = torch.cat([self.branch_1,self.branch_2,self.branch_3,self.branch_4],1)
        return x

In [44]:
class GoogleNet(nn.Module):
    def __init__(self,num_classes,aux_flag=True):
        super(GoogleNet,self).__init__()
        # input image 224*224
        self.conv1 = nn.Conv2d(3,64,kernel_size=7,stride=2,padding=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3,stride=2)
        self.conv2 = nn.Conv2d(64,192,kernel_size=3,stride=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3,stride=2)
        
        self.inception3a = InceptionV1(192,64,96,128,16,32,32)
        self.inception3b = InceptionV1(256,64,96,128,16,32,32)
        
        self.maxpool3 = nn.MaxPool2d(kernel_size=3,stride=2)
        
        self.inception4a = InceptionV1(480,192,96,208,16,48,64)
        
        self.inception4b = InceptionV1(512,160,112,224,24,64,64)
        self.inception4c = InceptionV1(512,128,128,256,24,64,64)
        self.inception4d = InceptionV1(512,112,144,288,32,64,64)
        self.inception4e = InceptionV1(528,256,160,320,32,128,128)
        
        self.maxpool4 = nn.MaxPool2d(kernel_size=3,stride=2)
        
        self.inception5a = InceptionV1(832,256,160,320,32,128,128)
        self.inception5b = InceptionV1(832,384,192,384,48,128,128)
        
        self.avgpool = nn.AvgPool2d(kernel_size=7,stride=1)
        self.dropout = nn.Dropout(p=0.4)
        self.linear = nn.Linear(1024,num_classes)
        
        if self.aux_flag:
            self.aux4a = Inception_aux(512,num_classes)
            self.aux4d = Inception_aux(528,num_classes)
        
    def forward(self,x):
        
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)
        x = self.inception4a(x)
        if self.aux_flag:
            aux1 = self.aux4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        if self.aux_flag:
            aux2 = self.aux4d(x)
        x = self.inception4e(x)
        x = self.maxpool4(x)
        x = self.inception5a(x)
        x = self.inception5b(x)
        x = self.avgpool(x)
        x = self.dropout(x)
        x = self.linear(x)
        
        if self.aux_flag:
            return x, aux1, aux2
        else:
            return x

In [45]:
net = GoogleNet(1000,True)

TypeError: __init__() got an unexpected keyword argument 'kenel_size'